In [3]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import random

import os

import dotenv

In [1]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.78 Safari/537.36 OPR/47.0.2631.39",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/603.2.4 (KHTML, like Gecko) Version/10.1.1 Safari/603.2.4",
    "My User Agent 1.0",
]

In [5]:
import requests
from bs4 import BeautifulSoup
import random
from pydantic import BaseModel
from typing import Optional
from datetime import date, datetime
import json


class BBCModel(BaseModel):
    url: str
    category: Optional[str] = "General"
    date: date
    header: Optional[str]
    body: Optional[str]


class BBC:

    def __init__(self, url, category="General"):
        self.url = url
        self.category = category
        self.date = date.today()
        self.header = None
        self.body = None
        self.soup = None
        self.scrape()

    def scrape(self):
        headers = {"User-Agent": random.choice(user_agents)}

        response = requests.get(self.url, headers=headers)
        self.soup = BeautifulSoup(response.content, "html.parser")
        self.header = self.get_header(self.soup)
        self.body = self.get_body(self.soup)

    def get_header(self, soup):
        # Adjust the CSS selector according to the actual HTML structure
        header_tag = soup.find("h1")
        if header_tag:
            return header_tag.get_text().strip()
        return None

    def get_body(self, soup):
        text_block_div = soup.find("div", {"data-component": "text-block"})
        if text_block_div:
            paragraphs = text_block_div.find_all("p")
            return " ".join([p.get_text().strip() for p in paragraphs])
        return None

    def get_url(self):
        return self.url

    def get_category(self):
        return self.category

    def to_json(self):
        date_datetime = datetime(self.date.year, self.date.month, self.date.day)
        return {
            "url": self.get_url(),
            "category": self.get_category(),
            "date": date_datetime,
            "header": self.get_header(self.soup),
            "body": self.get_body(self.soup),
        }

In [38]:
sjsu_page = "https://www.sjsu.edu/"

In [39]:
def scrape_page():
    print("Scraping Page...")
    url = "https://www.sjsu.edu/"
    headers = {"User-Agent": random.choice(user_agents)}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    with open("scraped_content.txt", "w", encoding="utf-8") as file:
        file.write(soup.get_text())

scrape_page()    

Scraping Page...


In [40]:
def scrape_links_from_page():
    print("Scraping Page...")
    url = "https://www.sjsu.edu/"
    headers = {"User-Agent": random.choice(user_agents)}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [a.get('href') for a in soup.find_all('a', href=True)]
    # return links
    print(len(links))
    # for link in links:
    #     print(link)
    return links
all_links= scrape_links_from_page()

Scraping Page...
161


In [41]:
def separate_links(links):
    http_links = [link for link in links if link.startswith('http')]
    non_http_links = [link for link in links if not link.startswith('http')]
    return http_links, non_http_links

http_links, non_http_links = separate_links(all_links)
print("HTTP Links: ", len(http_links))
print("Non-HTTP Links: ", len(non_http_links))

print("HTTP Links: ", non_http_links)


HTTP Links:  146
Non-HTTP Links:  15
HTTP Links:  ['#maincontent', '/', '/visit/index.php', '/admissions/index.php', '/giving/', '/online/index.php?utm_source=sjsumainmenu&utm_medium=web&utm_campaign=sjsuonline-navmenu-traffic', '/giving/', '/index.php', '/online/index.php?utm_source=sjsumainmenu-mobile&utm_medium=web&utm_campaign=sjsuonline-navmenu-traffic', '/about/', '', '/facts-and-accomplishments/', '/online/', 'tel:4089241000', '/online/index.php']


In [46]:
def add_prefix_to_links(links, prefix):
    return [prefix + link for link in links]

final_non_http_links = add_prefix_to_links(non_http_links, sjsu_page)

# print("Final Non-HTTP Links: ", final_non_http_links)


final_links = final_non_http_links + http_links
final_links = [link.replace("https://one.sjsu.edu", "").strip() for link in final_links]
final_links = [link for link in final_links if link]
# Save links to a text file
with open("final_links.txt", "w") as f:
    for link in final_links:
        f.write(link + "\n")

print("Final Links: ", len(final_links))

Final Links:  160


In [47]:
print("Final Links: ", final_links[59])

Final Links:  http://www.sjsu.edu/search/index.html


In [48]:
from tqdm import tqdm  # Import tqdm


def scrape_all_pages(links):
    if (links=="https://one.sjsu.edu"):
        return
    all_pages_content = []
    for link in tqdm(links, desc="Scraping Pages"):  # Wrap links with tqdm
        print("Scraping Page: ", link)
        if (link=="https://one.sjsu.edu"):
            return
        headers = {"User-Agent": random.choice(user_agents)}
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        all_pages_content.append(soup)
    return all_pages_content


all_pages_content = scrape_all_pages(final_links)

# Save the scraped content to a text file
with open("scraped_pages_content.txt", "w", encoding="utf-8") as file:
    for page in all_pages_content:
        file.write(page.get_text() + "\n\n")

Scraping Pages:   0%|          | 0/160 [00:00<?, ?it/s]

Scraping Page:  https://www.sjsu.edu/#maincontent


Scraping Pages:   1%|▏         | 2/160 [00:00<00:31,  4.96it/s]

Scraping Page:  https://www.sjsu.edu//
Scraping Page:  https://www.sjsu.edu//visit/index.php


Scraping Pages:   2%|▏         | 3/160 [00:00<00:25,  6.04it/s]

...

Scraping Pages:  98%|█████████▊| 157/160 [00:55<00:00,  5.00it/s]

Scraping Page:  https://www.sjsu.edu/up/resources/file-a-complaint.php


Scraping Pages:  99%|█████████▉| 159/160 [00:55<00:00,  4.93it/s]

Scraping Page:  https://www.sjsu.edu/titleixeo/make-a-report/index.php
Scraping Page:  https://a.cms.omniupdate.com/11/?skin=sjsu&account=sjsu&site=www&action=de&path=/index.php


Scraping Pages: 100%|██████████| 160/160 [00:56<00:00,  2.85it/s]


In [53]:
def clean_empty_lines(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as infile:
        lines = infile.readlines()

    # Remove empty lines and strip whitespace
    cleaned_lines = [line.strip() for line in lines if line.strip()]

    with open(output_file, "w", encoding="utf-8") as outfile:
        for line in cleaned_lines:
            outfile.write(line + "\n")


clean_empty_lines("scraped_pages_content.txt", "filtered_data.txt")

In [57]:
import re

# Load the text data
with open("scraped_pages_content.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Step 1: Remove boilerplate content by identifying common navigational patterns
patterns = [
    r"Skip to main content",
    r"Search",
    r"Visit",
    r"Apply",
    r"Give",  # Common phrases
    r"SJSU on [A-Za-z]+",
    r"SJSU Online",  # Social media links
    r"Footer",
    r"Last Updated .*",  # Footer and Last updated information
]
for pattern in patterns:
    text = re.sub(pattern, "", text)

# Step 2: Remove excessive whitespace
text = re.sub(r"\n\s*\n", "\n", text)  # Reduce multiple newlines to a single newline

# Step 3: Remove repetitive sections
# Example: We could split sections by keywords and check for duplicates
unique_sections = list(set(text.split("\n\n")))  # Split and deduplicate sections
cleaned_text = "\n\n".join(unique_sections)

# Step 4: Save the cleaned text
with open("cleaned_scraped_contentL1.txt", "w", encoding="utf-8") as file:
    file.write(cleaned_text)

In [58]:
import re


def clean_noisy_data(text):
    # Define a regex pattern to match unwanted characters
    # This pattern removes non-printable characters and any other specified noise
    cleaned_text = re.sub(
        r"[^\x20-\x7E]+", "", text
    )  # Keep only printable ASCII characters
    return cleaned_text


def clean_file(input_file, output_file):
    with open(input_file, "r", encoding="utf-8", errors="ignore") as infile:
        lines = infile.readlines()

    cleaned_lines = [clean_noisy_data(line) for line in lines]

    with open(output_file, "w", encoding="utf-8") as outfile:
        for line in cleaned_lines:
            if line.strip():  # Only write non-empty lines
                outfile.write(line + "\n")


# Usage
clean_file("cleaned_scraped_contentL1.txt", "cleaned_scraped_contentL2.txt")